In [108]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [109]:
df_tr_mcc_codes = pd.read_csv('./sources/tr_mcc_codes.csv', sep=';')
df_tr_types = pd.read_csv('./sources/tr_types.csv', sep=';')
df_transactions = pd.read_csv('./sources/transactions.csv', sep=',')
df_gender_train = pd.read_csv('./sources/gender_train.csv', sep=',')


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


1:

Выбранные строки типов транзакций

In [110]:
df_selected_tr_types = pd.DataFrame(df_transactions['tr_type'].sample(n=1000, random_state=12345));
df_selected_tr_types

,tr_type
4566725,2010
1804536,7031
1542649,1010
2143695,1010
2524759,1010
...,...
2343467,2010
5047120,2010
5884561,1010
2614021,7010


2:

Ключ / расшифровка транзакции

In [111]:
df_sorted_tr_key_value = df_tr_types.sort_values(by = ['tr_type'])
df_sorted_tr_key_value

,tr_type,tr_description
97,1000,Покупка.ТУ СБ РФ
98,1010,Покупка. POS ТУ СБ РФ
99,1030,Оплата услуги. Банкоматы СБ РФ
100,1100,Покупка. ТУ Россия
101,1110,Покупка. POS ТУ Россия
...,...,...
79,8151,Урегулирование безнадежной задолженности
80,8210,Плата за приост. действия карты
81,8220,Плата за обработку счета банковской карты клие...
82,8230,Плата за предоставление по запросу клиента док...


Массив выбранных типов

In [112]:

list_tr_types = list(set(list(df_selected_tr_types['tr_type'])))
list_tr_types.sort()
list_tr_types




[1010,
 1030,
 1100,
 1110,
 1200,
 1210,
 2010,
 2011,
 2330,
 2370,
 2371,
 4010,
 4011,
 4051,
 4071,
 4110,
 6100,
 6110,
 6200,
 7010,
 7011,
 7014,
 7020,
 7030,
 7031,
 7070,
 7071]

Получаем датафрейм выбранных типов транзакций и их расшифровок

In [113]:
list_exist_in_selected_tr_types = list(df_sorted_tr_key_value['tr_type'].apply(lambda x: x in list_tr_types))
df_filtered_by_tr_type_selection = df_sorted_tr_key_value.loc[list_exist_in_selected_tr_types];
df_filtered_by_tr_type_selection

,tr_type,tr_description
98,1010,Покупка. POS ТУ СБ РФ
99,1030,Оплата услуги. Банкоматы СБ РФ
100,1100,Покупка. ТУ Россия
101,1110,Покупка. POS ТУ Россия
102,1200,Покупка. Зарубеж. ТУ
103,1210,Покупка. POS Зарубеж. ТУ
106,2010,Выдача наличных в АТМ Сбербанк России
107,2011,Выдача наличных в АТМ Сбербанка (в других ТБ)
118,2330,Списание с карты по операции “перевода с карты...
129,2370,Списание с карты на карту по операции <перевод...


Получение описаний транзакций с наличием 'POS' и 'ATM'

In [114]:
import re
df_filtered_by_has_words = df_filtered_by_tr_type_selection.loc[df_filtered_by_tr_type_selection['tr_description'].apply(lambda x: bool(re.search("POS | АТМ", x)))]
df_filtered_by_has_words


,tr_type,tr_description
98,1010,Покупка. POS ТУ СБ РФ
101,1110,Покупка. POS ТУ Россия
103,1210,Покупка. POS Зарубеж. ТУ
106,2010,Выдача наличных в АТМ Сбербанк России
107,2011,Выдача наличных в АТМ Сбербанка (в других ТБ)
118,2330,Списание с карты по операции “перевода с карты...
5,4010,Плата за получение наличных в АТМ Сбербанка
6,4011,Плата за получение наличных в АТМ Сбербанка (в...
14,4051,Плата за перевод на карту (с карты) через АТМ ...
25,4110,Плата за получение наличных в АТМ. Россия


Доля найденных строк к выбранному числу транзакций (n / 1000)

In [115]:
df_filtered_by_has_words.shape[0] / 1000

0.016

## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

1:

Частота встречаемости значений в столбце tr_type

In [116]:
dictionary_value_repeats = {}
list_tr_type_values = list(df_transactions['tr_type'])

for x in list_tr_type_values:
    if x in dictionary_value_repeats:
        dictionary_value_repeats[x]+=1
    else:
        dictionary_value_repeats[x] = 1


dictionary_value_repeats

{1030: 970994,
 7010: 258088,
 2330: 69633,
 1010: 1589734,
 2010: 1040713,
 7020: 14529,
 1110: 928154,
 6110: 19822,
 7014: 2424,
 1100: 117870,
 2020: 13035,
 2370: 376521,
 7070: 691896,
 7030: 155200,
 7071: 96146,
 2331: 20881,
 4051: 20877,
 6010: 1947,
 2011: 63127,
 4011: 12337,
 4010: 32518,
 2320: 4765,
 2371: 65809,
 4071: 65806,
 4110: 9429,
 2110: 9457,
 2460: 1419,
 1200: 90742,
 2440: 3588,
 4090: 1021,
 1210: 20830,
 7021: 8663,
 7031: 34676,
 6200: 3855,
 6100: 1829,
 7074: 2457,
 4041: 884,
 4210: 3221,
 2210: 4372,
 7011: 11029,
 4021: 84,
 2021: 570,
 7040: 252,
 2340: 183,
 4031: 562,
 7034: 838,
 7035: 199,
 1410: 1302,
 1310: 1173,
 2456: 652,
 4097: 541,
 6000: 52,
 2100: 124,
 4100: 138,
 4500: 326,
 4061: 49,
 2341: 49,
 7024: 82,
 4020: 295,
 8145: 12,
 1510: 20,
 7075: 438,
 4200: 31,
 2200: 38,
 7041: 89,
 1000: 43,
 7015: 58,
 8146: 10,
 6210: 501,
 2446: 77,
 4096: 77,
 8100: 137,
 7025: 33,
 4045: 8,
 4035: 3,
 7044: 1,
 2000: 1}

2:

Топ 10 транзакций

In [117]:
sorted_array_of_tupples = sorted(dictionary_value_repeats.items(), key=lambda item: -item[1])

def get_most_popular_tr(mx, n):
    most_popular_tr = []
    index = 0
    for [key, value] in mx:
        most_popular_tr.append(key)
        index+=1
        if index == n:
            return most_popular_tr

array_most_popular_tr = get_most_popular_tr(sorted_array_of_tupples, 10)
array_most_popular_tr

[1010, 2010, 1030, 1110, 7070, 2370, 7010, 7030, 1100, 7071]

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

1:

In [118]:
df_customer_amount = df_transactions[['customer_id', 'amount']]
df_customer_amount

,customer_id,amount
0,39026145,-2245.92
1,39026145,56147.89
2,39026145,-56147.89
3,39026145,-1392.47
4,39026145,-920.83
...,...,...
6849341,61870738,-5176.84
6849342,61870738,-1652.77
6849343,61870738,-4687.23
6849344,61870738,-4491.83


In [119]:
df_customer_amount_positive = df_customer_amount.loc[df_customer_amount['amount'].apply(lambda x: True if x >= 0 else False)]
df_customer_amount_positive

,customer_id,amount
1,39026145,56147.89
16,39026145,224591.58
42,39026145,22459.16
67,39026145,22459.16
71,39026145,96574.38
...,...,...
6849238,61870738,18641.10
6849239,61870738,44918.32
6849256,61870738,89207.77
6849319,61870738,112295.79


In [120]:
df_customer_receive = df_customer_amount_positive.groupby('customer_id', group_keys=False).sum().sort_values(by = 'amount')
df_customer_receive

,amount
customer_id,
34419732,2.200000e-01
75010111,1.123000e+01
6766644,1.482000e+01
14267405,2.246000e+01
26965510,2.246000e+01
...,...
76310395,1.280732e+09
95303296,1.662873e+09
84219086,1.916438e+09


Id кастомера с наибольшим получением на карту за транзакции

In [121]:
customer_max_receive = df_customer_receive.tail(1).iloc[0].name
customer_max_receive

48281795

2:

In [122]:
df_customer_amount_negative = df_customer_amount.loc[df_customer_amount['amount'].apply(lambda x: True if x < 0 else False)]
df_customer_amount_negative

,customer_id,amount
0,39026145,-2245.92
2,39026145,-56147.89
3,39026145,-1392.47
4,39026145,-920.83
5,39026145,-14643.37
...,...,...
6849340,61870738,-4491.83
6849341,61870738,-5176.84
6849342,61870738,-1652.77
6849343,61870738,-4687.23


In [123]:
df_customer_send = df_customer_amount_negative.groupby('customer_id', group_keys=False).sum().sort_values(by = 'amount')
df_customer_send

,amount
customer_id,
48281795,-8.541879e+09
84219086,-1.904247e+09
87252636,-1.878556e+09
7999570,-1.866938e+09
81671241,-1.592583e+09
...,...
12403092,-1.706900e+03
68089681,-1.459850e+03
30237195,-1.122960e+03


Id кастомера с максимальной тратой по карте

In [124]:
customer_max_send = df_customer_send.head(1).iloc[0].name
customer_max_send

48281795

3:

Модуль разница между суммой приходов и суммой расходов найденных кастомеров

In [125]:
abs((abs(df_customer_receive.tail(1)['amount']) - abs(df_customer_send.head(1)['amount']))).iloc[0]

365148.4399738312

## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

1:

In [126]:
stat = df_transactions[df_transactions['tr_type'].isin(array_most_popular_tr)]['amount'].describe()
print('Среднее арифметическое: ', stat["mean"])
print('Медиана: ', stat['50%'])

Среднее арифметическое:  -16959.582910655463
Медиана:  -5311.59


2:

In [127]:
array_max_receive_tr = list(set(list(df_transactions[df_transactions['customer_id'] == customer_max_receive]['tr_type'])))
array_max_receive_tr

[2370, 7011, 7074, 1030, 2331, 4051, 7030, 2330, 2011, 7070, 4031]

In [128]:
array_max_send_tr = list(set(list(df_transactions[df_transactions['customer_id'] == customer_max_send]['tr_type'])))
array_max_send_tr

[2370, 7011, 7074, 1030, 2331, 4051, 7030, 2330, 2011, 7070, 4031]

Max receive customer:

In [129]:
stat = df_transactions[df_transactions['tr_type'].isin(array_max_receive_tr)]['amount'].describe()
print('Среднее арифметическое: ', stat["mean"])
print('Медиана: ', stat['50%'])

Среднее арифметическое:  -9083.125735868547
Медиана:  -2245.92


Max send customer:

In [130]:
stat = df_transactions[df_transactions['tr_type'].isin(array_max_send_tr)]['amount'].describe()
print('Среднее арифметическое: ', stat["mean"])
print('Медиана: ', stat['50%'])

Среднее арифметическое:  -9083.125735868547
Медиана:  -2245.92


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [131]:
transactions = df_transactions
transactions = pd.merge(transactions, df_tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, df_tr_types, how='inner')
transactions = pd.merge(transactions, df_gender_train, how='left')

transactions.shape

(6846580, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

1:

In [132]:
mean_amount_gender_1_send = transactions[(transactions['gender'] == 1) & (transactions['amount'] < 0)]['amount'].mean()
mean_amount_gender_1_send

-76586.11756644867

In [133]:
mean_amount_gender_0_send = transactions[(transactions['gender'] == 0) & (transactions['amount'] < 0)]['amount'].mean()
mean_amount_gender_0_send

-51838.07925577545

In [134]:
abs(abs(mean_amount_gender_0_send) - abs(mean_amount_gender_1_send))

24748.03831067322

2:

In [135]:
mean_amount_gender_1_receive = transactions[(transactions['gender'] == 1) & (transactions['amount'] >= 0)]['amount'].mean()
mean_amount_gender_1_receive

208102.17032507414

In [136]:
mean_amount_gender_0_receive = transactions[(transactions['gender'] == 0) & (transactions['amount'] >= 0)]['amount'].mean()
mean_amount_gender_0_receive

143017.99640496558

In [137]:
abs(abs(mean_amount_gender_0_receive) - abs(mean_amount_gender_1_receive))

65084.17392010856

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

1:

In [138]:
df_gender_1_send = transactions[(transactions['gender'] == 1) & (transactions['amount'] < 0)].groupby(by = 'mcc_code')['amount'].sum()
df_gender_1_send

mcc_code
742    -1.588169e+06
1711   -5.838790e+06
1731   -1.394321e+06
1799   -3.869702e+06
2741   -3.372876e+05
            ...     
8999   -1.465699e+08
9222   -6.526557e+06
9311   -2.412421e+06
9399   -1.696506e+07
9402   -6.154323e+05
Name: amount, Length: 183, dtype: float64

In [139]:
df_gender_0_send = transactions[(transactions['gender'] == 0) & (transactions['amount'] < 0)].groupby(by = 'mcc_code')['amount'].sum()
df_gender_0_send

mcc_code
742    -3046712.99
1711   -2487385.42
1731     -92037.63
1799   -4319703.88
2741    -155056.90
           ...    
9211    -822005.23
9222   -3590412.24
9311   -2029594.11
9399   -9255896.58
9402    -165100.86
Name: amount, Length: 184, dtype: float64

2:

In [140]:
df_mcc_codes_abs = (abs(abs(df_gender_1_send) - abs(df_gender_0_send)) / (abs(df_gender_1_send) + abs(df_gender_0_send))).fillna(0)
df_mcc_codes_abs_sorted = df_mcc_codes_abs.sort_values()[::-1].head(10).astype('int64')
df_mcc_codes_abs_sorted_top10 = list(df_mcc_codes_abs_sorted.index)
df_mcc_codes_abs_sorted_top10

[5940, 5968, 7531, 8699, 8398, 1731, 8220, 5072, 3351, 6211]

In [141]:
def createDict(df, field1, field2):
    dict = {}
    df_dict = df.to_dict()
    for x in df_dict[field1]:
        dict[df_dict[field1][x]] = df_dict[field2][x]

    return dict

In [142]:
df_top10_code_descr = transactions[['mcc_code', 'mcc_description']][transactions['mcc_code'].apply(lambda x: x in df_mcc_codes_abs_sorted_top10)]
df_top10_code_descr = df_top10_code_descr.drop_duplicates(subset = 'mcc_code')

dict = createDict(df_top10_code_descr, 'mcc_code', 'mcc_description')

def mapping(code):
    return dict[code]

data = {'mcc_code': df_mcc_codes_abs_sorted_top10, 'mcc_description': map(mapping, df_mcc_codes_abs_sorted_top10)}

pd.DataFrame(data)



,mcc_code,mcc_description
0,5940,Веломагазины — продажа и обслуживание
1,5968,Прямой маркетинг — торговые точки подписки
2,7531,Станции технического обслуживания для автомоби...
3,8699,"Организации, членства, нигде ранее не классифи..."
4,8398,Организации благотворительные и общественные с...
5,1731,Подрядчики по электричеству
6,8220,"Колледжи, университеты, профессиональные школы..."
7,5072,Оборудование и сопутствующие материалы для тех...
8,3351,Агентства по аренде автомобилей
9,6211,Ценные бумаги: брокеры/дилеры


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 